# Texas Cosmetologist Violations

Texas has a system for [searching for license violations](https://www.tdlr.texas.gov/cimsfo/fosearch.asp). You're going to search for cosmetologists!

> You can use the classwork notebook and also [my Everything scraping reference](https://jonathansoma.com/everything/scraping)

## Setup: Import what you'll need to scrape the page

We'll be using either Playwrightfor this, *not* requests.

In [1]:
from playwright.async_api import async_playwright
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)
page = await browser.new_page()
await page.goto("https://www.tdlr.texas.gov/cimsfo/fosearch.asp")

<Response url='https://www.tdlr.texas.gov/cimsfo/fosearch.asp' request=<Request url='https://www.tdlr.texas.gov/cimsfo/fosearch.asp' method='GET'>>

## Starting your search

Starting from [here](https://www.tdlr.texas.gov/cimsfo/fosearch.asp), search for cosmetologist violations for people with the last name **Nguyen**.

In [11]:
# click on "Search by License Program Type", and select value = 'COS'
await page.locator("#pht_status.mousebox").select_option('COS')

['COS']

In [12]:
await page.locator("#pht_lnm").fill("Nguyen")

In [17]:
await page.locator('input[type="submit"][name="B1"]').click()

## Scraping

Once you are on the results page, do this. **I step you through things bit by bit, so it's going to be a little different than we did in class.** Also, no `pd.read_html` allowed because this isn't reaaallly tabular data (or there's just too much to clean up).

Once you've loaded the page: **what selector are you going to do use to find the data?** In class we used things like `.article` or `.head-list`.

> You honestly can do this all with Playwright, no BeautifulSoup involved! But we didn't cover that in class so you should stick to this method:
> 
> ```python
> html = await page.content()
> doc = BeautifulSoup(html)
> ```

In [18]:
from bs4 import BeautifulSoup
html = await page.content()
doc = BeautifulSoup(html)

In [26]:
doc.find_all('tr')

[<tr><th style="padding:4px; text-align:left; background:#c2c2c2;">Name and Location</th><th style="padding:4px; text-align:left; background:#c2c2c2;">Order</th><th style="padding:4px; text-align:left; background:#c2c2c2;">Basis for Order</th></tr>,
 <tr style="background:#ffffff;"> <td style="padding:4px; text-align:left; font-size:11px; font:Arial, Helvetica, sans-serif; width:22%;"><span class="results_text">NGUYEN, THU THI </span><br/> <span class="default_text">City:</span> <span class="results_text">HOUSTON</span><br/> <span class="default_text">County:</span> <span class="results_text">HARRIS</span><br/> <span class="default_text">Zip Code:</span> <span class="results_text">77031</span><br/><br/><br/><span class="default_text"> License #:</span> <span class="results_text">816854</span><br/><br/><span class="default_text">Complaint #</span> <span class="results_text">COS20230009980</span></td> <td style="padding:4px; text-align:left; font-size:11px; font:Arial, Helvetica, sans-se

### Grab the rows, loop through each result and print the entire row

It's probably too many rows to do all at once: once you have a list, use `[:10]` to only show the first ten! For example, if you saved the table rows into `results` you might do something like this:

```python
for result in results[:10]:
    print("------")
    print(result)
```

Although you'd want to print out the text from the row (I give example output below).

In [31]:
results = doc.find_all('tr')
for result in results[:10]:
    print("------")
    print(result.text)

------
Name and LocationOrderBasis for Order
------
 NGUYEN, THU THI  City: HOUSTON County: HARRIS Zip Code: 77031 License #: 816854Complaint # COS20230009980 Date: 10/23/2023Respondents Thu Thi Nguyen and Tien Minh Vo are assessed an administrative penalty in the amount of $1,350. Respondents failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondents failed to discard single use implements after each use; Respondents failed to keep floors, walls, ceilings, shelves, furniture, furnishings, and fixtures clean and in good repair; Respondents failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used; Respondents failed to clean, disinfect, and sterilize metal instruments with a Department-approved sterilizer in accordance with the sterilizer or sanitizers manufacturer's instructions.
------
 NGUYEN, TUONG  City: FORT WORTH County: TARRANT Zip Code: 76120 License #: 742271Complaint 

The result should look something like this:

```
Name and Location Order Basis for Order
NGUYEN, THANH
City: FRISCO
County: COLLIN
Zip Code: 75034


License #: 790672

Complaint # COS20210004784 Date: 11/16/2021

Respondent is assessed an administrative penalty in the amount of $1,875. Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.
NGUYEN, LONG D
City: SAN SABA
County: SAN SABA
Zip Code: 76877
```

### Loop through each result and print each person's name

You'll probably get an error because the first one doesn't have a name. How do you make that not happen?! If you want to ignore an error, you use code like this:

```python
try:
   # try to do something
except:
   print("It didn't work')
```

It should help you out. If you don't want to print anything when there's an error, you can type `pass` instead of the `print` statement.

**Why doesn't the first one have a name?**

Output should look like this:

```
Doesn't have a name
NGUYEN, THANH
NGUYEN, LONG D
NGUYEN, LUCIE HUONG
NGUYEN, CHINH
NGUYEN, JIMMY
```

* *Tip: The name has a class you can use. The class name is reused in a lot of places, but because it's the first one you don't have to worry about that!*
* *Tip: Instead of searching across the entire page – like `doc.select_one` – you should be doing your searching just inside of each **row** (I used this technique in the beginning of class with BeautifulSoup when we were scraping the books page)* 

In [35]:
for result in results[:10]:
    try:
        print(result.select_one('.results_text').text)
    except:
        print("Doesn't have a name")

Doesn't have a name
NGUYEN, THU THI 
NGUYEN, TUONG 
NGUYEN, HUONG T
NGUYEN, STEVEN QUOC 
NGUYEN, HUONG THI NGOC 
NGUYEN, TAMMY 
NGUYEN, TOAN DUC 
NGUYEN, NHAN TRONG 
NGUYEN, TAM PHUONG 


## Loop through each result, printing each violation description ("Basis for order")

Your results should look something like:

```
Doesn't have a violation
Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.
Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used, the Department is charging 2 violations; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to clean and disinfect manicure tables prior to use for each client.
...
```

> - *Tip: You'll get an error even if you're ALMOST right - which row is causing the problem?*
> - *Tip: Or I guess you could just skip the one with the problem using try/except...*

In [36]:
for result in results[:10]:
    try:
        print(result.select('[style="padding:4px; text-align:left; font-size:11px; font:Arial, Helvetica, sans-serif; width:39%;"]')[-1].text)
    except:
        print("Doesn't have a violation")

Doesn't have a violation
Respondents failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondents failed to discard single use implements after each use; Respondents failed to keep floors, walls, ceilings, shelves, furniture, furnishings, and fixtures clean and in good repair; Respondents failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used; Respondents failed to clean, disinfect, and sterilize metal instruments with a Department-approved sterilizer in accordance with the sterilizer or sanitizers manufacturer's instructions.
Respondents employed an individual as an operator who had not obtained a cosmetology license; Respondents failed to follow whirlpool foot spas cleaning and sanitization procedures as required, the Department is charging 2 violations; Respondents failed to clean, disinfect, and sterilize metal instruments with a Department-approved sterilizer in accordance

## Loop through each result, printing the complaint number

Output should look similar to this:

```
Doesn't have a complaint number
COS20210004784
COS20210009745
COS20210011484
...
```

- *Tip: Think about the order of the elements. Can you count from the opposite direction than you normally do?*

In [38]:
for result in results[:10]:
    try:
        print(result.select('.results_text')[-2].text)
    except:
        print("Doesn't have a complaint number")

Doesn't have a complaint number
COS20230009980
COS20230001960
COS20220017901
COS20220018135
COS20220016107
COS20220013443
COS20220015714
COS20220012646
COS20220012645


## Saving the results

### Loop through each result to create a list of dictionaries

Each dictionary must contain

- Person's name
- Violation description
- Violation number
- License Numbers
- Zip Code
- County
- City

Create a new dictionary for each result (except the header).

Based on what you print out, the output might look something like:

```
This row is broken: Name and Location Order Basis for Order
{'name': 'NGUYEN, THANH', 'city': 'FRISCO', 'county': 'COLLIN', 'zip_code': '75034', 'complaint_no': 'COS20210004784', 'license_numbers': '790672', 'complaint': 'Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.'}
{'name': 'NGUYEN, LONG D', 'city': 'SAN SABA', 'county': 'SAN SABA', 'zip_code': '76877', 'complaint_no': 'COS20210009745', 'license_numbers': '760420, 1620583', 'complaint': 'Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used, the Department is charging 2 violations; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to clean and disinfect manicure tables prior to use for each client.'}
```

> _**Tip:** Depending on how you do this, you might want to figure out how to search for "next siblings" or "following siblings"_

In [51]:
all_row = []
for result in results:
    try:
        row = {
            'name': result.select('.results_text')[0].text,
            'city': result.select('.results_text')[1].text,
            'county': result.select('.results_text')[2].text,
            'zip_code': result.select('.results_text')[3].text,
            'license_numbers': result.select('.results_text')[-3].text,
            'complaint_no': result.select('.results_text')[-2].text,
            'complaint': result.select('[style="padding:4px; text-align:left; font-size:11px; font:Arial, Helvetica, sans-serif; width:39%;"]')[-1].text
        }
        print(row)
        all_row.append(row)
    except:
        print("This row is broken: Name and Location Order Basis for Order")

This row is broken: Name and Location Order Basis for Order
{'name': 'NGUYEN, THU THI ', 'city': 'HOUSTON', 'county': 'HARRIS', 'zip_code': '77031', 'license_numbers': '816854', 'complaint_no': 'COS20230009980', 'complaint': "Respondents failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondents failed to discard single use implements after each use; Respondents failed to keep floors, walls, ceilings, shelves, furniture, furnishings, and fixtures clean and in good repair; Respondents failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used; Respondents failed to clean, disinfect, and sterilize metal instruments with a Department-approved sterilizer in accordance with the sterilizer or sanitizers manufacturer's instructions."}
{'name': 'NGUYEN, TUONG ', 'city': 'FORT WORTH', 'county': 'TARRANT', 'zip_code': '76120', 'license_numbers': '742271', 'complaint_no': 'COS20230001960', 

### Save that to a CSV named `output.csv`

The dataframe should look something like...

|index|name|city|county|zip_code|complaint_no|license_numbers|complaint|
|---|---|---|---|---|---|---|---|
|0|NGUYEN, THANH|FRISCO|COLLIN|75034|COS20210004784|790672|Respondent failed to clean and sanitize whirlp...|
|1|NGUYEN, LONG D|SAN SABA|SAN SABA|76877|COS20210009745|760420, 1620583|Respondent failed to keep a record of the date...|


- *Tip: If you send a list of dictionaries to `pd.DataFrame(...)`, it will create a dataframe out of that list!*

In [52]:
import pandas as pd
table = pd.DataFrame(all_row[1:])

In [53]:
table.head(10)

,name,city,county,zip_code,license_numbers,complaint_no,complaint
0,"NGUYEN, TUONG",FORT WORTH,TARRANT,76120,742271,COS20230001960,Respondents employed an individual as an opera...
1,"NGUYEN, HUONG T",SAN ANTONIO,BEXAR,78259,775126,COS20220017901,Respondent failed to clean and disinfect facia...
2,"NGUYEN, STEVEN QUOC",SAN ANGELO,TOM GREEN,76904,"748707, 1458935",COS20220018135,Respondent failed to cooperate with the inspec...
3,"NGUYEN, HUONG THI NGOC",MANSFIELD,TARRANT,76063,829081,COS20220016107,"Respondent failed to clean, disinfect, and ste..."
4,"NGUYEN, TAMMY",TOMBALL,HARRIS,77375,789659,COS20220013443,Respondents employed an unlicensed individual ...
5,"NGUYEN, TOAN DUC",SUGAR LAND,FORT BEND,77478,789406,COS20220015714,"Respondent failed to clean, disinfect, and ste..."
6,"NGUYEN, NHAN TRONG",SAN ANTONIO,BEXAR,78247,"694927, 1096417",COS20220012646,Respondent failed to follow whirlpool foot spa...
7,"NGUYEN, TAM PHUONG",SAN ANTONIO,BEXAR,78259,"731908, 828834",COS20220012645,Respondent failed to follow whirlpool foot spa...
8,"NGUYEN, NHUNG NGOC THI",ALLEN,COLLIN,75002,758450,COS20230010084,Respondent failed to keep a record of the date...
9,"NGUYEN, ANH NGOC",HUFFMAN,HARRIS,77336,795350,COS20230010404,"Respondent failed to clean, disinfect, and ste..."


### Open the CSV file and examine the first few. Make sure you didn't save an extra weird unnamed column.

In [55]:
table.to_csv("Texas_Cosmetology_Violations.csv", index=False)

In [ ]:
# When having multiple names, it only saves the first person